In [1]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [2]:
Podsumowanie_paczek_i_Pw("24/48")

OBIĘTOSC PACZEK (Z OWATAMI)
21/01: 162m3
WST: 128m3
Objetość wszystkich paczek: 290m3
PW 24/48 spakowano: 197M3


In [3]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"])
# br.to_excel("BRAKI.xlsx")

,OPIS,WOLNE_SALDO,SALDO,PACZKA,DATA_KOMPLETACJI,ZAMOWIONE,UWAGI,GRUPA
0,AVANT 2][,-1.0,0.0,21/01,2024-07-10,0.0,...,1
2,GOYA 1,-4.0,0.0,WST,2024-07-17,20.0,"PIANKI ZAMOWIONE P, , 2024-07-05,",2
3,"GOYA 1,5",-6.0,0.0,WST,2024-07-17,20.0,"PIANKI ZAMOWIONE P, , 2024-07-05,",2
6,GOYA N,-4.0,0.0,WST,2024-07-17,20.0,"PIANKI ZAMOWIONE P, , 2024-07-05,",2
8,GOYA P112x100,-2.0,0.0,WST,2024-07-17,10.0,"PIANKI ZAMOWIONE P, , 2024-07-05,",2
9,GOYA P112x80,-2.0,0.0,WST,2024-07-17,10.0,"PIANKI ZAMOWIONE P, , 2024-07-05,",2
4,GOYA [1,-8.0,0.0,WST,2024-07-17,30.0,"PIANKI ZAMOWIONE P, , 2024-07-05,",2
5,"GOYA [1,5",-8.0,0.0,WST,2024-07-17,30.0,"PIANKI ZAMOWIONE P, , 2024-07-05,",2
7,GOYA [L/[LH,-4.0,0.0,WST,2024-07-17,20.0,"PIANKI ZAMOWIONE P, , 2024-07-05,",2
1,LENOX [1H],-1.0,18.0,WST,2024-07-17,120.0,"PIANKI ZAMOWIONE P, , 2024-07-05,",2


In [4]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

WOLNE PONIZEJ MIN: 60 POZYCJE
WOLNE PONIZEJ MIN NIE ZAMOWIONE: 41 POZYCJE
SALDO PONIZEJ MIN: 47 POZYCJE
SALDO PONIZEJ MIN NIE ZAMOWIONE: 33 POZYCJE


,OPIS,ZAMOWIONE,CZEKA_NA_SPAKOWANIE,SALDO,MIN,WOLNE_SALDO
0,AVANT [1],0.0,0.0,0.0,2,0.0
8,"AVANT 2,5",0.0,0.0,3.0,5,2.0
19,AVANT LM][,0.0,0.0,8.0,15,8.0
20,AVANT P104x64,0.0,0.0,9.0,10,7.0
21,AVANT P112x84,0.0,0.0,10.0,12,9.0
22,AVANT P112x97,0.0,0.0,20.0,21,20.0
34,"AMALFI 1,5 MEMORY",0.0,0.0,28.0,30,20.0
40,AMALFI [LA MEMORY,0.0,0.0,7.0,8,6.0
50,CALYPSO E80D,0.0,0.0,7.0,10,2.0
69,DIVA N,0.0,0.0,1.0,2,0.0


In [5]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

MAX_obj:                    3263
ZAMOWIONE_obj:              606
CZEKA_NA_SPAKOWANIE_obj:    22
SALDO_obj:                  2242
WOLNE_obj:                  1953
WOLNE_NIE_SPAK_obj:         1975
DO_ZAM_obj:                 688
---
ZAPEŁNIENIE MAGAZYNU
SALDO / MAX: 68.7%
WOLNE / MAX: 59.8%
WOLNE_NIE_SPAK / MAX: 60.5%
ZAPEŁNIENIE MAG PO ZDJECIU 1 PACZEK RAZEM Z ZAMOWIENIAMI: 79.1%


In [6]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp, instrukcja_zamawiania_ciech as izc
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)



In [7]:
arp

OBJ_CIECH       45.96
OBJ_VITA        33.19
OBJ_PIANPOL    372.85
RAZEM          452.00

In [9]:
arp["HORIZON"].ar

,OPIS,ZAMOWIONE,CZESIOWO_DOSTARCZONE,CZEKA_NA_SPAKOWANIE,SALDO,SALDO_Z_NIE_SPAK,MIN,MAX,SUMA_ZLEC,WOLNE_SALDO,WOLNE_NIE_SPAK,21/01,WST,DO_ZAM_SZT
108,HORIZON 1z,0.0,150.0,0.0,189.0,189.0,160,320.0,32.0,157.0,157.0,24.0,8.0,13.0
109,"HORIZON 1,5z",0.0,150.0,0.0,252.0,252.0,200,400.0,21.0,231.0,231.0,15.0,6.0,19.0
110,"HORIZON 2,5z",0.0,25.0,0.0,59.0,59.0,40,80.0,5.0,54.0,54.0,3.0,2.0,1.0
111,"HORIZON T2,5z",0.0,0,0.0,1.0,1.0,2,5.0,0.0,1.0,1.0,0,0.0,4.0
112,HORIZON T3z,0.0,0,0.0,2.0,2.0,2,4.0,0.0,2.0,2.0,0,0.0,2.0
113,"HORIZON T3,5z",0.0,0,0.0,5.0,5.0,4,8.0,0.0,5.0,5.0,0,0.0,3.0
114,HORIZON Onz,0.0,0,0.0,28.0,28.0,18,35.0,1.0,27.0,27.0,0,1.0,8.0
115,HORIZON LHz][,0.0,30.0,0.0,85.0,85.0,70,140.0,15.0,70.0,70.0,10.0,5.0,40.0
116,HORIZON L2Hz][,0.0,50.0,0.0,85.0,85.0,70,140.0,7.0,78.0,78.0,5.0,2.0,12.0
117,HORIZON 117Nz,0.0,0,0.0,14.0,14.0,6,12.0,1.0,13.0,13.0,1.0,0.0,0.0


In [8]:
arp.Tabela_podsumowania_analizy

,GRUPA,MODEL,BRYL_DO_ZAMOWIENIA,WSPL_DO_ZAM,OBJ_CIECH,OBJ_VITA,OBJ_PIANPOL,*WSPL_ZAP_WOLNE,*WSPL_ZAP_ZAM
1,1,ELIXIR,117.0,31.0,0.00,0.00,50.24,44.0,69.0
0,1,AVANT,255.0,29.0,0.00,0.00,64.47,75.0,75.0
3,1,HORIZON,114.0,12.0,0.00,33.19,6.91,57.0,57.0
2,1,STONE,36.0,4.0,0.00,0.00,9.80,54.0,101.0
4,2,AMALFI,141.0,25.0,0.00,0.00,50.22,77.0,77.0
9,2,REVERSO,137.0,22.0,0.00,0.00,40.04,60.0,80.0
5,2,DIVA,54.0,21.0,0.00,0.00,16.44,84.0,84.0
6,2,MAXWELL,27.0,16.0,0.00,0.00,9.66,76.0,92.0
8,2,WILLOW,51.0,14.0,0.00,0.00,20.87,58.0,89.0
7,2,OXYGEN,27.0,12.0,0.00,0.00,7.91,96.0,96.0


In [ ]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"



In [ ]:
# Zapis_danych_do_Archiwum(242X, analiza, arp._Tabela_podsumowania_analizy, "sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db")